In [1]:
%matplotlib inline

import sys, os
sys.path.append("/home/daniil/repos/pytorch-segmentation-detection/")
sys.path.insert(0, '/home/daniil/repos/pytorch-segmentation-detection/vision/')

# Use second GPU -pytorch-segmentation-detection- change if you want to use a first one
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

from PIL import Image
from matplotlib import pyplot as plt

import torch
from torchvision import transforms
from torch.autograd import Variable
import pytorch_segmentation_detection.models.resnet_dilated as resnet_dilated

import numpy as np
import imageio

valid_transform = transforms.Compose(
                [
                     transforms.ToTensor(),
                     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                ])


fcn = resnet_dilated.Resnet18_8s(num_classes=2)
fcn.load_state_dict(torch.load('resnet_18_8s_binary_best.pth'))
fcn.cuda()
fcn.eval()

test_videos_name_template = 'Segmentation/Dataset{}/Video.avi'
#video_submission_file_template = 'binary_submissions/Dataset{0}/{1:08d}.png'

video_submission_file_template = 'binary_submissions/Dataset{0}/Segmentation.avi'
fps = 25

datasets_numbers = xrange(1, 7)

for current_dataset_number in datasets_numbers:

    current_images_video_filename = test_videos_name_template.format(current_dataset_number)
    current_video_submission_filename = video_submission_file_template.format(current_dataset_number)

    # Creating readers for each of our videos
    images_reader = imageio.get_reader(current_images_video_filename, 'ffmpeg')

    current_dataset_number_of_images = images_reader.get_length()
    
    writer = imageio.get_writer(current_video_submission_filename, format='tiff')#, ffmpeg_params=['-crf', '0'])

    for index in xrange(current_dataset_number_of_images):

        current_image = np.asarray(images_reader.get_data(index))

        img_not_preprocessed = Image.fromarray(current_image).convert('RGB')

        img = valid_transform(img_not_preprocessed)

        img = img.unsqueeze(0)

        img = Variable(img.cuda())

        res = fcn(img)

        _, hard_labels = res.squeeze(0).max(0)

        segmentation = hard_labels.data.cpu().numpy().squeeze()

        segmentation[segmentation > 0] = 80
        segmentation = segmentation[:, :, None].repeat(3, axis=2)
        segmentation = segmentation.astype(np.uint8)
        
        writer.append_data(segmentation)

#         save_image_filename = video_submission_file_template.format(current_dataset_number, index)

#         plt.imsave(save_image_filename, segmentation)

    writer.close()



/home/daniil/repos/anaconda2/lib/python2.7/site-packages/torch/nn/modules/module.py:482: UserWarning: src is not broadcastable to dst, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  own_state[name].copy_(param)
/home/daniil/repos/anaconda2/lib/python2.7/site-packages/torch/nn/functional.py:1423: UserWarning: nn.functional.upsample_bilinear is deprecated. Use nn.functional.upsample instead.
  warnings.warn("nn.functional.upsample_bilinear is deprecated. Use nn.functional.upsample instead.")
